In [ ]:
geo = gpd.read_file('../Datasets/ultimate_move_df.shp')
queen_w = Queen.from_shapefile("../Datasets/ultimate_move_df.shp")
n, bins, patches = plt.hist(queen_w.cardinalities.values(), 25, facecolor='green', alpha=0.75)

Identifying Unusual Block Groups Based on Neighbor Counts

Now that the adjacency matrix is built, we can examine some characteristics of the links formed. Below we plot a histogram of the number of neighbors. As you can see, the median number of neighbors for these units is somewhere around 50. Two oddities that we can see here are that there is one block group with 0 neighbors and one block group with 175(!) neighbors. We examine these further below.

In [ ]:
plt.xlabel('Number of Neighbors')
plt.ylabel('Count')
plt.title('Group Connectivity: Queen Adjacency Neighbors')
plt.grid(True)

We can easily identify the “island” block group by selecting for the block group with 0 neighbors. Given that this is not a typical area within Spain, we will drop it from the analysis. However, in general, areas with no neighbors could still be included in a spatial regression model, although they would receive no spatial weight and therefore would be defined as having no relationship to any other geographic units.

From the histogram you can also see that there is a huge outlier - a block group that has 175 neighbors. We plot it below.

In [ ]:
idmax=pd.Series(queen_w.cardinalities).idxmax()
p=geo.loc[[idmax]].plot()
s=p.set_xticks([])
s=p.set_yticks([])
title=plt.title('Area with 175 Neighbours')

In [ ]:
new dataset

In [ ]:
geo = gpd.read_file('../Datasets/ultimate_move_df_cleaned.shp')

In [ ]:
df_tryout = geo.copy(deep=True)
df_tryout['tot_build_perc'] = df_tryout.srf_tot/df_tryout.shape__are
df_tryout['housing_perc'] = df_tryout.srf_housin/df_tryout.shape__are

df_tryout['shape__are'] = df_tryout.shape__are /1000000
df_tryout['pop_per_km2'] = df_tryout.tot_pop/df_tryout.shape__are
df_tryout['male_female_ratio'] = df_tryout.Male/df_tryout.Female
df_tryout['young_per_km2'] = df_tryout.sub_16_age / df_tryout.shape__are
df_tryout['middle_per_km2'] = df_tryout['16_to_64_a'] / df_tryout.shape__are
df_tryout['old_per_km2'] = df_tryout['64_more_ag'] / df_tryout.shape__are
df_tryout['travel_perc'] = df_tryout.pob_sale/df_tryout.tot_pop
df_tryout['stay_home_perc'] = df_tryout.pob_casa/df_tryout.tot_pop

df_tryout = df_tryout.drop(['shape_leng','shape_leng','NO2_avg_ut'
                           ,'sub_16_age','16_to_64_a','64_more_ag','weight_urb','weight_tr','Male',
                            'Female','srf_housin','srf_tot'], axis = 1)

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(16, 12))

import seaborn as sns
mask = np.zeros_like(df_tryout.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(df_tryout.corr(), mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

seed=7

kfold=KFold(n_splits=10, random_state=seed, shuffle = True)

num_trees=100
num_features=5
max_depth = 5
model=RandomForestRegressor(n_estimators=num_trees, max_features=num_features,max_depth=max_depth, random_state=seed)
model.fit(X,NO2_avg)

results_NO2_avg=cross_val_score(model, X, NO2_avg, cv=kfold)

print(f'Random Forest - Accuracy {results_NO2_avg.mean()*100:.3f}% std {results_NO2_avg.std()*100:3f}%')

res_w1["Res"]=results_NO2_avg
res_w1["Type"]="Random Forest"

resall=pd.concat([resall,res_w1], ignore_index=True)

NameError: name 'X' is not defined

In [ ]:
mean_NO2_avg = NO2_avg.mean()
print(f'Average NO2 level: {mean_NO2_avg}')
print()
print(f'Expected error range: [{mean_NO2_avg-(1-results_NO2_avg.mean()/100)}, {mean_NO2_avg+(1-results_NO2_avg.mean()/100)}]')

In [ ]:
plt.figure(figsize=(40,9))

for name, importance in zip(col_names, model.feature_importances_):
    print(f'{name:15s}  {importance:.4f}')

sns.barplot(x=col_names, y=model.feature_importances_, order=col_names)

In [ ]:
df_tryout['less45m2_per_km2'] = (df_tryout['less_30_m2'] + df_tryout['30_45_m2'])/df_tryout.shape__are
df_tryout['46m2_50m2_per_km2']=df_tryout['46_60_m2']/df_tryout.shape__are
df_tryout['61m2_90m2_per_km2'] = (df_tryout['61_75_m2'] + df_tryout['76_90_m2'] )/df_tryout.shape__are
df_tryout['more90m2_per_km2'] = ( df_tryout['91_105_m2']+df_tryout['106_120_m2'] +  df_tryout['121_150_m2'] + df_tryout['150_180_m2'] + df_tryout['above_180_'])/df_tryout.shape__are


df_tryout = df_tryout.drop(['less_30_m2' ,'30_45_m2','46_60_m2', '61_75_m2','76_90_m2','91_105_m2','106_120_m2','121_150_m2','150_180_m2',
                            'above_180_','first_home', 'pop_per_km2','tot_house','stay_home_perc','pob_casa','male_female_ratio','second_hom','vacation_h'], axis = 1)

In [ ]:
plt.figure(figsize=(30,9))

for name, importance in zip(col_names, model.feature_importances_):
    print(f'{name:15s}  {importance:.4f}')

sns.barplot(x=col_names, y=model.feature_importances_)

In [ ]:
predict = model.predict(X)

abs_differ = abs(predict - NO2_avg)
abs_differ = pd.DataFrame(abs_differ)

abs_differ = abs_differ.merge(df_tryout.geometry, how = 'left', left_index = True, right_index = True)
abs_differ = gpd.GeoDataFrame(abs_differ, crs="EPSG:4326", geometry='geometry')


vmin, vmax = 0, 6
fig, ax = plt.subplots(1, figsize=(30, 20))
abs_differ.plot(column='NO2_avg', cmap='gist_yarg', linewidth=0.8,ax = ax, legend=True)

In [ ]:
Y = df.NO2_avg
income = df.Income
pob_sale = df.pob_sale
travel_perc = df.travel_perc

tot_build_perc = df.tot_build_perc
housing_perc = df.housing_perc

young_per_km2 = df.young_per_km2
middle_per_km2 = df.middle_per_km2
old_per_km2 = df.old_per_km2

less45m2_per_km2 = df.less45m2_per_km2
med_small_per_km2 = df['46m2_50m2_per_km2']
large_med_per_km2 = df['61m2_90m2_per_km2']
more90m2_per_km2 = df.more90m2_per_km2

####################
income = np.asarray(income).reshape(-1,1).astype(float)
pob_sale = np.asarray(pob_sale).reshape(-1,1).astype(float)
travel_perc = np.asarray(travel_perc).reshape(-1,1).astype(float)

tot_build_perc = np.asarray(tot_build_perc).reshape(-1,1).astype(float)
housing_perc = np.asarray(housing_perc).reshape(-1,1).astype(float)

young_per_km2 = np.asarray(young_per_km2).reshape(-1,1).astype(float)
middle_per_km2 = np.asarray(middle_per_km2).reshape(-1,1).astype(float)
old_per_km2 = np.asarray(old_per_km2).reshape(-1,1).astype(float)

less45m2_per_km2 = np.asarray(less45m2_per_km2).reshape(-1,1).astype(float)
med_small_per_km2 = np.asarray(med_small_per_km2).reshape(-1,1).astype(float)
large_med_per_km2 = np.asarray(large_med_per_km2).reshape(-1,1).astype(float)
more90m2_per_km2 = np.asarray(more90m2_per_km2).reshape(-1,1).astype(float)

######################
y_lag = ps.weights.lag_spatial(queen_w, Y)
income_lag = ps.weights.lag_spatial(queen_w, income)
pob_sale_lag = ps.weights.lag_spatial(queen_w, pob_sale)
travel_perc_lag = ps.weights.lag_spatial(queen_w, travel_perc)

tot_build_perc_lag = ps.weights.lag_spatial(queen_w, tot_build_perc)
housing_perc_lag = ps.weights.lag_spatial(queen_w, housing_perc)

young_per_km2_lag = ps.weights.lag_spatial(queen_w, young_per_km2)
middle_per_km2_lag = ps.weights.lag_spatial(queen_w, middle_per_km2)
old_per_km2_lag = ps.weights.lag_spatial(queen_w, old_per_km2)

less45m2_per_km2_lag = ps.weights.lag_spatial(queen_w, less45m2_per_km2)
med_small_per_km2_lag = ps.weights.lag_spatial(queen_w, med_small_per_km2)
large_med_per_km2_lag = ps.weights.lag_spatial(queen_w, large_med_per_km2)
more90m2_per_km2_lag = ps.weights.lag_spatial(queen_w, more90m2_per_km2)



lag_matrix = np.concatenate(( income_lag,  pob_sale_lag,travel_perc_lag, tot_build_perc_lag, housing_perc_lag, young_per_km2_lag, middle_per_km2_lag, old_per_km2_lag, less45m2_per_km2_lag, med_small_per_km2_lag, large_med_per_km2_lag,more90m2_per_km2_lag),axis=1)
names= ['income_lag','pob_sale_lag','travel_perc_lag','tot_build_perc_lag','housing_perc_lag','young_per_km2_lag','middle_per_km2_lag','old_per_km2_lag','less45m2_per_km2_lag','med_small_per_km2_lag','large_med_per_km2_lag','more90m2_per_km2_lag']

lag_df = pd.DataFrame(data=lag_matrix, columns= names, index = None )
lag_df.head()

In [ ]:
from esda.moran import Moran
moran_Y  = Moran(Y , queen_w)
from splot.esda import moran_scatterplot
fig, ax = moran_scatterplot(moran_Y , aspect_equal=True)
plt.show()

In [ ]:
income_lag1 = pd.DataFrame(income_lag)
income_lag1 = income_lag1.merge(df_tryout.geometry, how = 'left', left_index = True, right_index = True)
income_lag1 = gpd.GeoDataFrame(income_lag1, crs="EPSG:4326", geometry='geometry')
income_lag1

In [ ]:
vmin, vmax = 3000, 12000

fig, ax = plt.subplots(1, figsize=(30, 20))

income_lag1.plot(column=0, cmap='gist_yarg', linewidth=0.8,ax = ax)

In [ ]:
resall =pd.DataFrame()
res_w1 =pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

seed=7

kfold=KFold(n_splits=10, random_state=seed, shuffle = True)

num_trees=100
num_features=5
max_depth = 5
model=RandomForestRegressor(n_estimators=num_trees, max_features=num_features,max_depth=max_depth, random_state=seed)
model.fit(lag_matrix,Y)

results_NO2_avg=cross_val_score(model, lag_matrix, Y, cv=kfold)

print(f'Random Forest - Accuracy {results_NO2_avg.mean()*100:.3f}% std {results_NO2_avg.std()*100:3f}%')

res_w1["Res"]=results_NO2_avg
res_w1["Type"]="Random Forest"

resall=pd.concat([resall,res_w1], ignore_index=True)

In [ ]:
plt.figure(figsize=(30,9))

for name, importance in zip(names, model.feature_importances_):
    print(f'{name:15s}  {importance:.4f}')

sns.barplot(x=names, y=model.feature_importances_)

In [ ]:
pred = model.predict(lag_matrix)

abs_diff =  abs(pred - Y)
abs_diff = pd.DataFrame(abs_diff)

abs_diff = abs_diff.merge(df_tryout.geometry, how = 'left', left_index = True, right_index = True)

abs_diff = gpd.GeoDataFrame(abs_diff, crs="EPSG:4326", geometry='geometry')

vmin, vmax = 0, 3

fig, ax = plt.subplots(1, figsize=(30, 20))

abs_diff.plot(column='NO2_avg', cmap='gist_yarg', linewidth=0.8,ax = ax)

In [ ]:
mix_matrix = np.concatenate(( income_lag,  pob_sale_lag, less45m2_per_km2_lag, med_small_per_km2, large_med_per_km2,more90m2_per_km2,young_per_km2_lag,housing_perc),axis=1)
names= ['income_lag','pob_sale_lag','less45m2_per_km2','med_small_per_km2','large_med_per_km2','more90m2_per_km2','young_per_km2_lag','housing_perc']



In [ ]:
mix_matrix1= pd.DataFrame(mix_matrix)
mix_matrix1.columns = names
f, ax = plt.subplots(figsize=(16, 12))

mask = np.zeros_like(mix_matrix1.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(mix_matrix1.corr(), mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
resall =pd.DataFrame()
res_w1 =pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

seed=7

kfold=KFold(n_splits=10, random_state=seed, shuffle = True)

num_trees=100
num_features=5
max_depth = 5
model1=RandomForestRegressor(n_estimators=num_trees, max_features=num_features,max_depth=max_depth, random_state=seed)
model1.fit(mix_matrix,Y)

results_NO2_avg=cross_val_score(model1, mix_matrix, Y, cv=kfold)

print(f'Random Forest - Accuracy {results_NO2_avg.mean()*100:.3f}% std {results_NO2_avg.std()*100:3f}%')

res_w1["Res"]=results_NO2_avg
res_w1["Type"]="Random Forest"

resall=pd.concat([resall,res_w1], ignore_index=True)

In [ ]:
mean_NO2_avg = Y.mean()
print(f'Average NO2 level: {mean_NO2_avg}')
print()
print(f'Expected error range: [{mean_NO2_avg-(1-results_NO2_avg.mean()/100)}, {mean_NO2_avg+(1-results_NO2_avg.mean()/100)}]')

In [ ]:
plt.figure(figsize=(30,9))

for name, importance in zip(names, model1.feature_importances_):
    print(f'{name:15s}  {importance:.4f}')

sns.barplot(x=names, y=model1.feature_importances_)

In [ ]:
pred1 = model1.predict(mix_matrix)

abs_diff1 = abs(Y-pred1)/Y
abs_diff1 = pd.DataFrame(abs_diff1)

abs_diff1 = abs_diff1.merge(df_tryout.geometry, how = 'left', left_index = True, right_index = True)
abs_diff1

abs_diff1 = gpd.GeoDataFrame(abs_diff1, crs="EPSG:4326", geometry='geometry')

vmin, vmax = 0, 0.5
#fig, ax = plt.subplots(1, figsize=(30, 20))

#abs_diff1.plot(column='NO2_avg', cmap='gist_yarg', linewidth=0.8,ax = ax)

In [ ]:
resall =pd.DataFrame()
res_w1 =pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

seed=7

kfold=KFold(n_splits=10, random_state=seed, shuffle = True)

num_trees=100
num_features=5
max_depth = 5
model12=xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 
model12.fit(mix_matrix,Y)

results_NO2_avg=cross_val_score(model12, mix_matrix, Y, cv=kfold)

print(f'Random Forest - Accuracy {results_NO2_avg.mean()*100:.3f}% std {results_NO2_avg.std()*100:3f}%')

res_w1["Res"]=results_NO2_avg
res_w1["Type"]="Random Forest"

resall=pd.concat([resall,res_w1], ignore_index=True)

In [ ]:
plt.figure(figsize=(30,9))

for name, importance in zip(names, model12.feature_importances_):
    print(f'{name:15s}  {importance:.4f}')

sns.barplot(x=names, y=model12.feature_importances_)

In [ ]:
pred2 = model12.predict(mix_matrix)

abs_diff2 = abs(Y-pred2)/Y
abs_diff2 = pd.DataFrame(abs_diff2)

abs_diff2 = abs_diff2.merge(df_tryout.geometry, how = 'left', left_index = True, right_index = True)
abs_diff2

abs_diff2 = gpd.GeoDataFrame(abs_diff2, crs="EPSG:4326", geometry='geometry')

#vmin, vmax = 0, 1
#ig, ax = plt.subplots(1, figsize=(30, 20),vmin =0, vmax =  1)

#abs_diff2.plot(column='NO2_avg', cmap='gist_yarg', linewidth=0.8,ax = ax, legend=True)

In [ ]:
abs_diff2.NO2_avg = np.clip(abs_diff2.NO2_avg, 0, 1)
fig, ax = plt.subplots(1, figsize=(30, 20))
abs_diff2.plot(column='NO2_avg', cmap='gist_yarg', linewidth=0.8,ax = ax, legend=True)